In [14]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json
from time import sleep
import os
import random
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import Point
import time
import numpy as np
from matplotlib.colors import TwoSlopeNorm, Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import folium
from branca.colormap import linear
import statsmodels.api as sm
import statsmodels.formula.api as smf
import branca.colormap as cm
import statsmodels.formula.api as smf

## Load matched pairs dataframe

In [36]:
filtered_df2 = pd.read_csv('CityMatchedPOIsCleanedforDID.csv')
filtered_df2

,Original POI ID,Original Avg Pre Pre Open,Original Avg Pre Open,Original Avg Post Open,POI ID,Zip_Code,Group,Station,Open Date,City,...,sim_longitude,ev_latitude,ev_longitude,ev_level2,ev_dcfc,ev_type,Original POI Open Date,Original POI Open Year,Distance(m),distance_group
0,venue:aee02a68618c6a864e5fda30,1745.413043,1638.945652,1725.868132,venue:f68c3ea870d6dcf138dbc578,10475,Apparel,Baychester Retail III LLC,2/4/2022,Bronx,...,-73.829147,40.863231,-73.828839,0,12,dcfc,9/30/2019,2019,181.989541,100-200m
1,venue:b886e727ee41e117f7a2a34d,2084.315217,1734.489130,2685.384615,venue:f68c3ea870d6dcf138dbc578,10475,Apparel,Baychester Retail III LLC,2/4/2022,Bronx,...,-73.829147,40.863231,-73.828839,0,12,dcfc,9/30/2019,2019,76.171406,0-100m
2,venue:280d777332c37342dae138d2,15.000000,10.271739,10.130435,venue:1391dbb186a5427b7b5d6c06,10469,Apparel,Popeyes,1/14/2023,Bronx,...,-73.848907,40.880142,-73.907385,4,0,level2,8/28/2019,2019,211.675179,200-300m
3,venue:81c4527a932c30002cf37305,46.336957,48.282609,41.681319,venue:b9c1f667de33b9b05fb0e1e3,10469,Beauty & Spa,Popeyes,1/14/2023,Bronx,...,-73.845482,40.879501,-73.903603,8,0,level2,9/15/2022,2022,41.792961,0-100m
4,venue:5ba8b5e4f5ec44796e7de4ba,16.543478,23.891304,25.641304,venue:62139fec27f9195ec6cdbe4f,10462,Beauty & Spa,2244 Westchester Ave,12/26/2023,Bronx,...,-73.852501,40.882337,-73.880265,2,0,level2,11/25/2021,2021,339.725604,300-400m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,venue:168d5092d06927ae5b112849,52.741573,31.228261,33.652174,venue:885be026b56897f7a8579238,10101,Shops & Services,Hylan New Dorp Plaza,5/27/2022,Staten Island,...,-74.110985,40.588091,-74.089936,2,0,level2,8/28/2019,2019,218.493685,200-300m
1729,venue:65606e3cedc4ff54049bb7a2,19.539326,18.478261,13.793478,venue:885be026b56897f7a8579238,10101,Shops & Services,Hylan New Dorp Plaza,5/27/2022,Staten Island,...,-74.110985,40.588091,-74.089936,2,0,level2,8/28/2019,2019,35.633187,0-100m
1730,venue:5affc2b9507ae7ce1e574c25,40.269663,42.815217,55.326087,venue:885be026b56897f7a8579238,10101,Shops & Services,Hylan New Dorp Plaza,5/27/2022,Staten Island,...,-74.110985,40.588091,-74.089936,2,0,level2,8/28/2019,2019,335.678240,300-400m
1731,venue:d743c4576e85ee96195932bf,13.123596,6.847826,10.619565,venue:885be026b56897f7a8579238,10101,Shops & Services,Hylan New Dorp Plaza,5/27/2022,Staten Island,...,-74.110985,40.588091,-74.089936,2,0,level2,8/28/2019,2019,44.223606,0-100m


In [40]:
long_df = pd.DataFrame({
    'poi_id': filtered_df2['Original POI ID'].tolist() * 2 + filtered_df2['POI ID'].tolist() * 2,
    'group': ['original'] * len(filtered_df2) * 2 + ['similar'] * len(filtered_df2) * 2,
    'time': ['pre'] * len(filtered_df2) + ['post'] * len(filtered_df2) + ['pre'] * len(filtered_df2) + ['post'] * len(filtered_df2),
    'visits': filtered_df2['Original Avg Pre Open'].tolist() + filtered_df2['Original Avg Post Open'].tolist() + filtered_df2['Avg Pre Open'].tolist() + filtered_df2['Avg Post Open'].tolist(),
    'city': filtered_df2['City'].tolist() *2 + filtered_df2['City'].tolist() * 2,
    'category': filtered_df2['Group'].tolist() * 2 + filtered_df2['Group'].tolist() * 2,
    'Similar_POI_Zip_Code': filtered_df2['Zip_Code'].tolist() * 2 + filtered_df2['Zip_Code'].tolist() * 2,
    'distance_group': filtered_df2['distance_group'].tolist() * 2 + filtered_df2['distance_group'].tolist() * 2,
    'year': filtered_df2['Original POI Open Year'].tolist() * 4
})

In [42]:
long_df['groupbinary']= long_df['group'].apply(lambda x:1 if x=='original' else 0)
long_df['timebinary']= long_df['time'].apply(lambda x:1 if x=='post' else 0)
long_df

,poi_id,group,time,visits,city,category,Similar_POI_Zip_Code,distance_group,year,groupbinary,timebinary
0,venue:aee02a68618c6a864e5fda30,original,pre,1638.945652,Bronx,Apparel,10475,100-200m,2019,1,0
1,venue:b886e727ee41e117f7a2a34d,original,pre,1734.489130,Bronx,Apparel,10475,0-100m,2019,1,0
2,venue:280d777332c37342dae138d2,original,pre,10.271739,Bronx,Apparel,10469,200-300m,2019,1,0
3,venue:81c4527a932c30002cf37305,original,pre,48.282609,Bronx,Beauty & Spa,10469,0-100m,2022,1,0
4,venue:5ba8b5e4f5ec44796e7de4ba,original,pre,23.891304,Bronx,Beauty & Spa,10462,300-400m,2021,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6927,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,200-300m,2019,0,1
6928,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1
6929,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,300-400m,2019,0,1
6930,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1


In [44]:
long_df

,poi_id,group,time,visits,city,category,Similar_POI_Zip_Code,distance_group,year,groupbinary,timebinary
0,venue:aee02a68618c6a864e5fda30,original,pre,1638.945652,Bronx,Apparel,10475,100-200m,2019,1,0
1,venue:b886e727ee41e117f7a2a34d,original,pre,1734.489130,Bronx,Apparel,10475,0-100m,2019,1,0
2,venue:280d777332c37342dae138d2,original,pre,10.271739,Bronx,Apparel,10469,200-300m,2019,1,0
3,venue:81c4527a932c30002cf37305,original,pre,48.282609,Bronx,Beauty & Spa,10469,0-100m,2022,1,0
4,venue:5ba8b5e4f5ec44796e7de4ba,original,pre,23.891304,Bronx,Beauty & Spa,10462,300-400m,2021,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6927,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,200-300m,2019,0,1
6928,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1
6929,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,300-400m,2019,0,1
6930,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1


In [46]:
long_df['groupbinary']= long_df['group'].apply(lambda x:1 if x=='original' else 0)
long_df['timebinary']= long_df['time'].apply(lambda x:1 if x=='post' else 0)
long_df

,poi_id,group,time,visits,city,category,Similar_POI_Zip_Code,distance_group,year,groupbinary,timebinary
0,venue:aee02a68618c6a864e5fda30,original,pre,1638.945652,Bronx,Apparel,10475,100-200m,2019,1,0
1,venue:b886e727ee41e117f7a2a34d,original,pre,1734.489130,Bronx,Apparel,10475,0-100m,2019,1,0
2,venue:280d777332c37342dae138d2,original,pre,10.271739,Bronx,Apparel,10469,200-300m,2019,1,0
3,venue:81c4527a932c30002cf37305,original,pre,48.282609,Bronx,Beauty & Spa,10469,0-100m,2022,1,0
4,venue:5ba8b5e4f5ec44796e7de4ba,original,pre,23.891304,Bronx,Beauty & Spa,10462,300-400m,2021,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6927,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,200-300m,2019,0,1
6928,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1
6929,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,300-400m,2019,0,1
6930,venue:885be026b56897f7a8579238,similar,post,27.032609,Staten Island,Shops & Services,10101,0-100m,2019,0,1


## Vanila DID algorithm

In [48]:
model = smf.ols('visits ~ (groupbinary) : (timebinary) + C(year) +  C(poi_id)', data=long_df).fit()

In [49]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 visits   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     76.84
Date:                Tue, 20 May 2025   Prob (F-statistic):               0.00
Time:                        21:24:03   Log-Likelihood:                -43248.
No. Observations:                6932   AIC:                         9.049e+04
Df Residuals:                    4936   BIC:                         1.041e+05
Df Model:                        1995                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

## Heterogeneous treatment effects

In [53]:
distance_groups = ['0-100m', '100-200m', '200-300m', '300-400m']

In [55]:
for group in distance_groups:
    long_df[f'distance_group_{group.replace("-", "_").replace("m", "")}'] = (long_df['distance_group'] == group).astype(int)

In [61]:
poi_categories = ['Apparel', 'Beauty & Spa', 'Buildings and Entertainment Centers', 'Dining', 'Groceries','Home Improvements & Furnishings','Hotels & Casinos', 'Leisure','Medical & Health','Shopping Centers', 'Shops & Services']

In [63]:
for category in poi_categories:
    long_df[f'category_{category.replace(" ", "_").replace("&", "and")}'] = (long_df['category'] == category).astype(int)

In [69]:
model_distance = smf.ols('visits ~ (groupbinary) : (timebinary) :(distance_group_0_100) + (groupbinary) : (timebinary) : (distance_group_100_200) + (groupbinary) : (timebinary) : (distance_group_200_300)+ (groupbinary) : (timebinary):(distance_group_300_400) + C(year) + C(poi_id)', 
                data=long_df).fit()

In [70]:
print(model_distance.summary())

                            OLS Regression Results                            
Dep. Variable:                 visits   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     76.69
Date:                Tue, 20 May 2025   Prob (F-statistic):               0.00
Time:                        21:27:13   Log-Likelihood:                -43248.
No. Observations:                6932   AIC:                         9.049e+04
Df Residuals:                    4933   BIC:                         1.042e+05
Df Model:                        1998                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [82]:
model_cat = smf.ols('visits ~ (groupbinary) : (timebinary) :(category_Apparel) + (groupbinary) : (timebinary) : (category_Beauty_and_Spa) + (groupbinary) : (timebinary) : (category_Buildings_and_Entertainment_Centers)+ (groupbinary) : (timebinary):(category_Dining)+ (groupbinary) : (timebinary) :(category_Groceries) + (groupbinary) : (timebinary) :(category_Home_Improvements_and_Furnishings)  + (groupbinary) : (timebinary) :(category_Hotels_and_Casinos)  + (groupbinary) : (timebinary) :(category_Leisure)  + (groupbinary) : (timebinary) :(category_Medical_and_Health)  + (groupbinary) : (timebinary) :(category_Shopping_Centers)  + (groupbinary) : (timebinary) :(category_Shops_and_Services)   + C(year) + C(poi_id) + C(category) + C(city)', 
                data=long_df).fit()

In [84]:
print(model_cat.summary())

                            OLS Regression Results                            
Dep. Variable:                 visits   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     76.96
Date:                Tue, 20 May 2025   Prob (F-statistic):               0.00
Time:                        21:31:45   Log-Likelihood:                -43220.
No. Observations:                6932   AIC:                         9.045e+04
Df Residuals:                    4926   BIC:                         1.042e+05
Df Model:                        2005                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [90]:
import sys
import numpy as np
import pandas as pd
import matplotlib
import statsmodels
import geopandas as gpd

In [92]:
print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Statsmodels version:", statsmodels.__version__)
print("Geopandas version:", gpd.__version__)

Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
NumPy version: 1.26.4
Pandas version: 2.2.2
Matplotlib version: 3.9.2
Statsmodels version: 0.14.2
Geopandas version: 1.0.1
